# Raster dataset examples

Written by ChatGPT.

## GeoTiffs with GCPs and Tiepoints

In [ ]:
import numpy as np
import rasterio
from rasterio.control import GroundControlPoint
from rasterio.transform import Affine

# Define file path for output GeoTIFF
output_tiff = "output_with_gcps_tiepoints.tif"

# Create a dummy raster (100x100) with random values
width, height = 100, 100
data = np.random.randint(0, 255, (1, height, width), dtype=np.uint8)

# Define Affine transformation (default identity)
transform = Affine.translation(0, 0) * Affine.scale(1, -1)

# Define Ground Control Points (GCPs)
gcps = [
    GroundControlPoint(row=10, col=20, x=-122.123, y=37.456, z=50),
    GroundControlPoint(row=30, col=40, x=-122.456, y=37.789, z=60),
    GroundControlPoint(row=50, col=60, x=-122.789, y=38.123, z=55),
]

# Define Tie Points (Model Tiepoints)
tie_points = [
    (0, 0, 0, -122.1, 37.4, 0),
    (50, 50, 0, -122.2, 37.5, 0),
]

# Create a new GeoTIFF with GCPs
with rasterio.open(
    output_tiff,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,
    dtype=rasterio.uint8,
    crs="EPSG:4326",  # Define coordinate reference system (WGS84)
    transform=transform,
) as dst:
    dst.write(data)

    # Add Ground Control Points
    dst.gcps = (gcps, dst.crs)

    # Add Tie Points as metadata
    dst.update_tags(TIEPOINTS=str(tie_points))

print(f"GeoTIFF with GCPs and Tie Points saved as {output_tiff}")

In [ ]:
import xarray as xr

xr.open_dataset("output_with_gcps_tiepoints.tif", engine="rasterio")

In [ ]:
import rasterio

# Open the saved GeoTIFF
output_tiff = "output_with_gcps_tiepoints.tif"

with rasterio.open(output_tiff) as dataset:
    print("GeoTransform (Affine Transformation):")
    print(dataset.transform)
    print(dataset.gcps)
    print(dataset.tags())

## GDAL netCDF with GeoTransform

rasterio cannot write to netCDF

In [ ]:
import numpy as np
from osgeo import gdal, osr

# Define the output netCDF file
output_file = "output.nc"

# Define raster dimensions
cols = 10  # Number of columns
rows = 10  # Number of rows
bands = 1  # Number of bands

# Define geotransform (Origin X, Pixel Width, Rotation, Origin Y, Rotation, Pixel Height)
geotransform = [100.0, 30.0, 0.0, 200.0, 0.0, -30.0]  # Example values

# Define a spatial reference (WGS84 in this case)
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)  # WGS84
proj_wkt = srs.ExportToWkt()

# Create a new netCDF file using GDAL
driver = gdal.GetDriverByName("netCDF")
dataset = driver.Create(output_file, cols, rows, bands, gdal.GDT_Float32)

# Set geotransform and projection
dataset.SetGeoTransform(geotransform)
dataset.SetProjection(proj_wkt)

# Create some dummy data (a simple gradient)
data = np.arange(cols * rows, dtype=np.float32).reshape(rows, cols)

# Write data to the first band
band = dataset.GetRasterBand(1)
band.WriteArray(data)
band.SetNoDataValue(-9999)  # Set a no-data value

# Flush and close dataset
band.FlushCache()
dataset = None

print(f"NetCDF file '{output_file}' created with geotransform and projection.")

In [ ]:
import xarray as xr

xr.open_dataset("output.nc")